In [14]:
import tensorflow as tf
import pandas as pd
import nltk, re, time
from langdetect import detect
from contractions import get_contractions
from sqlalchemy import create_engine
from pprint import pprint
from nltk.corpus import stopwords
import chars2vec
import sklearn.decomposition
import matplotlib.pyplot as plt
import itertools
import string
import re
from keras.models import model_from_json
from keras.models import load_model
import gensim
import os
import numpy as np
from sklearn.utils import resample
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from nltk.tokenize import word_tokenize
from sklearn.utils import class_weight
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, GRU, GlobalMaxPool1D
from keras.layers.embeddings import Embedding
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import optimizers
from sklearn.metrics import roc_auc_score
from keras.initializers import Constant

# We download stopwords package
# nltk.download('stopwords')
contractions = get_contractions()


In [17]:
'''
1. Load the Steam review data into Pandas dataframe
'''

engine = create_engine('mysql://root:@localhost:3306/steam')
steam_data_query = """SELECT url AS reviewid, content, CAST(recommend AS SIGNED) AS recommend
    FROM latest_review WHERE url LIMIT 40000;"""
# test_data_query = """SELECT url AS reviewid, content, CAST(recommend AS SIGNED) AS recommend
#     FROM latest_review WHERE reviewid NOT IN (SELECT TOP 40000 url AS reviewid
#     FROM latest_review)"""
df = pd.read_sql(steam_data_query, engine)
test_data = pd.read_sql(test_data_query, engine)



/Users/admin/anaconda3/lib/python3.6/site-packages/sqlalchemy/engine/default.py:507: Warning: (1292, "Truncated incorrect INTEGER value: 'http://steamcommunity.com/id/--C10H15N/recommended/218620/'")
  cursor.execute(statement, parameters)
/Users/admin/anaconda3/lib/python3.6/site-packages/sqlalchemy/engine/default.py:507: Warning: (1292, "Truncated incorrect INTEGER value: 'http://steamcommunity.com/id/--Candy--/recommended/105450/'")
  cursor.execute(statement, parameters)
/Users/admin/anaconda3/lib/python3.6/site-packages/sqlalchemy/engine/default.py:507: Warning: (1292, "Truncated incorrect INTEGER value: 'http://steamcommunity.com/id/--HeLp---MePLiiS/recommended/252950/'")
  cursor.execute(statement, parameters)
/Users/admin/anaconda3/lib/python3.6/site-packages/sqlalchemy/engine/default.py:507: Warning: (1292, "Truncated incorrect INTEGER value: 'http://steamcommunity.com/id/--mihawk--/recommended/304050/'")
  cursor.execute(statement, parameters)
/Users/admin/anaconda3/lib/pytho

ProgrammingError: (_mysql_exceptions.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '40000 url AS reviewid\n    FROM latest_review)' at line 2") [SQL: 'SELECT url AS reviewid, content, CAST(recommend AS SIGNED) AS recommend\n    FROM latest_review WHERE reviewid NOT IN (SELECT TOP 40000 url AS reviewid\n    FROM latest_review)'] (Background on this error at: http://sqlalche.me/e/f405)

In [4]:

review_lines = list()
lines = df['content'].values.tolist()

for line in lines:   
    tokens = word_tokenize(line)
    # convert to lower case
    tokens = [w.lower() for w in tokens]
    # remove punctuation from each word    
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    words = [word for word in stripped if word.isalpha()]
    # filter out stop words    
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    review_lines.append(words)

In [6]:
len(review_lines)

91321

In [ ]:
EMBEDDING_DIM = 100
# train word2vec model
model = gensim.models.Word2Vec(sentences=review_lines, size=EMBEDDING_DIM, window=5, workers=4, min_count=1)
# vocab size
words = list(model.wv.vocab)
print('Vocabulary size: %d' % len(words))

In [ ]:
# save model in ASCII (word2vec) format
filename = 'steam_embedding_word2vec.txt'
model.wv.save_word2vec_format(filename, binary=False)